# Recomendação de Filmes

## 1. Importando as Bibliotecas

In [152]:
!pip install surprise

In [153]:
import pandas as pd
import numpy as np

from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVDpp
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy


## 2. Carregando Dados

In [227]:
links = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/links.csv')
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [228]:
movies = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [229]:
ratings = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [230]:
tags = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/tags.csv')
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


## 3. EDA

In [231]:
# Juntando as informações
filmes = ratings.join(movies.set_index('movieId'), on='movieId')

In [232]:
print('Tamanho da Base de Filmes: ', filmes.shape)

Tamanho da Base de Filmes:  (100836, 6)


In [233]:
filmes.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [161]:
print('Quantidade de Filmes Avaliados: ',
filmes['movieId'].value_counts().shape[0])

print('Quantidade de Usuários Avaliando: ',
filmes['userId'].value_counts().shape[0])

Quantidade de Filmes Avaliados:  9724
Quantidade de Usuários Avaliando:  610


In [162]:
# Quantidade de Avaliações TOP5 Filmes
filmes['title'].value_counts().head()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: title, dtype: int64

In [163]:
# Quantidade de Avaliações LOW5 Filmes
filmes['title'].value_counts().tail()

Hip Hop Witch, Da (2000)            1
Battle in Seattle (2007)            1
Cage Dive (2017)                    1
Disgrace (2008)                     1
What Men Still Talk About (2011)    1
Name: title, dtype: int64

In [164]:
# Quantidade de Avaliações TOP5 Usuários
filmes['userId'].value_counts().head()

414    2698
599    2478
474    2108
448    1864
274    1346
Name: userId, dtype: int64

In [165]:
# Quantidade de Avaliações LOW5 Usuários
filmes['userId'].value_counts().tail()

406    20
595    20
569    20
431    20
442    20
Name: userId, dtype: int64

In [262]:
# Avaliações do usuário 414
filmes.query('userId == 414').head()

,userId,movieId,rating,timestamp,title,genres
62294,414,1,4.0,961438127,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
62295,414,2,3.0,961594981,Jumanji (1995),Adventure|Children|Fantasy
62296,414,3,4.0,961439278,Grumpier Old Men (1995),Comedy|Romance
62297,414,5,2.0,961437647,Father of the Bride Part II (1995),Comedy
62298,414,6,3.0,961515642,Heat (1995),Action|Crime|Thriller


In [263]:
# Avaliações do usuário 414
filmes.query('userId == 414 and movieId == 168248')

,userId,movieId,rating,timestamp,title,genres


## 4. Modelo

In [367]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.2)

In [168]:
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

### 4.1 Treinamento do Modelo

In [169]:
knn = KNNBaseline(k=33, sim_options=sim_options)
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [170]:
slo = SlopeOne()
slo.fit(trainset)

In [171]:
svd = SVDpp(n_epochs=10, verbose=True)
svd.fit(trainset)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9


In [368]:
co = CoClustering(n_epochs=10, verbose=True)
co.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


### 4.2 Avaliação do Modelo

In [173]:
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.8765


0.8764541519882564

In [174]:
predictions_svd = svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 0.8718


0.871776960406083

In [345]:
predictions_co = co.test(testset)
accuracy.rmse(predictions_co)

RMSE: 1.5011


1.5010697628039456

In [176]:
predictions_slo = slo.test(testset)
accuracy.rmse(predictions_slo)

RMSE: 0.9002


0.9001668760437501

### 4.3 Predição do Modelo

Parameters:	
* uid – The (raw) user id. See this note.
* iid – The (raw) item id. See this note.
* r_ui (float) – The true rating rui.
* est (float) – The estimated rating r^ui.
* details (dict) – Stores additional details about the prediction that might be useful for later analysis.

In [346]:
# Predição com os dados de teste
predictions_co[:5]

[Prediction(uid=534, iid=46578, r_ui=2.5, est=4.0010697628039456, details={'was_impossible': False})]

In [255]:
# uid=247, iid=364 Conversão para o id interno
print(co.trainset.to_inner_uid(247))
print(co.trainset.to_inner_iid(364))

# uid=247, iid=364 Conversão para o id externo
print(co.trainset.to_raw_uid(300))
print(co.trainset.to_raw_iid(515))

300
515
247
364


#### 4.3.1. Função Análise de Recomendação

In [299]:
def recomenda_filme(userId,movieId):
  # ID do usuário para predição
  uid = userId
  # ID do filme para predição
  iid = movieId 
  nome_filme = movies.query('movieId == @movieId')['title'].values[0]
  print('Filme:', nome_filme)
  print('Usuário:', userId)
  if filmes.query('userId == @userId and movieId == @movieId')['title'].values.size == 0:
    print('Usuário não avaliou o filme!')
  else:
    nota_filme = ratings.query('userId == @userId and movieId == @movieId')['rating'].values[0]
    print('Avaliação do usuário:', nota_filme)
  # Predição baseada no melhor modelo
  print('Estimativa de Avaliação[0-5]:', round(co.predict(co.trainset.to_raw_uid(uid), co.trainset.to_raw_iid(iid))[3], 2))

In [303]:
recomenda_filme(406,3)

Filme: Grumpier Old Men (1995)
Usuário: 406
Usuário não avaliou o filme!
Estimativa de Avaliação[0-5]: 3.15


In [364]:
recomenda_filme(247,364) 

Filme: Lion King, The (1994)
Usuário: 247
Avaliação do usuário 5.0
Estimativa de Avaliação[0-5]: 3.06


#### 4.3.2. Função TOP Recomendações

In [472]:
def top_n(userId,n):
  # Selecionando apenas os filmes do treinamento
  lista_filmes_treino = []
  for x in trainset.all_items():
    lista_filmes_treino.append(trainset.to_raw_iid(x))
  # Selecionando os filmes do treinamento que o usuário não avaliou
  filmes_user = ratings.query('userId == @userId')['movieId'].values
  filmes_user_nao = movies.query('movieId not in @filmes_user')
  filmes_user_nao = filmes_user_nao.query('movieId in @lista_filmes_treino')['movieId'].values
  # Criando um ranking para o usuário para os filmes não avaliados
  ranking=[]
  for movieId in filmes_user_nao:
    ranking.append((movieId, co.predict(co.trainset.to_inner_uid(userId), co.trainset.to_inner_iid(movieId))[3]))
  # Ordenando os TOP filmes avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids dos filmes
  x,_ = zip(*ranking[:n])
  # Listando os nomes dos filmes em ordem de recomendação
  return movies.query('movieId in @x')['title'].copy().reset_index(drop=True)

In [473]:
top_n(414,5)

0                               Out to Sea (1997)
1                                  Prancer (1989)
2                           Big Tease, The (1999)
3    Jesus of Montreal (Jésus de Montréal) (1989)
4                                Session 9 (2001)
Name: title, dtype: object

In [474]:
top_n(406,5)

0                              Mute Witness (1994)
1    Farewell My Concubine (Ba wang bie ji) (1993)
2                          To Catch a Thief (1955)
3                             Graduate, The (1967)
4                                    Carrie (1976)
Name: title, dtype: object